In [5]:
import os
os.environ["OPENAI_API_KEY"]="                   "

In [6]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from  langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [7]:
loader = PyPDFLoader("coatings-07-00063.pdf")
documents = loader.load_and_split()

In [8]:
documents

[Document(metadata={'source': 'coatings-07-00063.pdf', 'page': 0}, page_content='coatings\nReview\nFabrication of Photovoltaic Textiles\nRobert R. Mather1,2,*,†and John I. B. Wilson1,2,†\n1Power Textiles Limited, Upland House, Ettrick Road, Selkirk TD7 5AJ, UK; j.i.b.wilson@hw.ac.uk\n2School of Engineering & Physical Sciences, Heriot-Watt University, Riccarton, Edinburgh EH14 4AS, UK\n*Correspondence: mather44robert@gmail.com; Tel.: +44-7747-792-357\n† These authors contributed equally to this work.\nAcademic Editor: Mahbubul Hassan\nReceived: 13 December 2016; Accepted: 21 April 2017; Published: 29 April 2017\nAbstract: Solar photovoltaic (PV) arrays are providing an increasing fraction of global electrical\ndemand, with an accelerating rate of new installations. Most of these employ conventional\nglass-fronted panels, but this type of PV array does not satisfy applications that require a light-weight,\nﬂexible PV generator. An option discussed in this article is to consider textiles 

In [22]:
len(documents)

40

In [24]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts=text_splitter.split_documents(documents)

##embeddings

In [27]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

##DB

In [30]:
db = Chroma.from_documents(texts,hf)

##Retriever

In [33]:
retriever=db.as_retriever(search_kwargs={"k":3})

##make a chain

In [36]:
qa_chain = RetrievalQA.from_llm(
    llm=chat,retriever=retriever
)

In [37]:
query="what is the main idea in this pdf"
response=qa_chain(query)

In [38]:
response

{'query': 'what is the main idea in this pdf',
 'result': 'The main idea in the provided context seems to be about different methods and approaches to making textile fabrics electrically conducting for various applications, such as photovoltaic cells or adding conductivity through metal or conductive polymer deposition onto fibers. It also discusses the challenges and solutions related to ensuring good conductivity without compromising the bulk fiber properties.'}

In [39]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [43]:
from langchain_community.vectorstores import FAISS

In [44]:
db_1=FAISS.from_documents(texts,embeddings)

In [45]:
retriever=db_1.as_retriever()

In [46]:
qa_chain_1= RetrievalQA.from_llm(
    llm=chat,retriever=retriever
)

In [47]:
query="what is the main idea in this pdf"
response=qa_chain_1(query)

In [48]:
response

{'query': 'what is the main idea in this pdf',
 'result': 'The main idea in the PDF seems to be exploring the innovative use of textiles as supports for electricity-generating photovoltaic (PV) solar cells. The document discusses different approaches to utilizing textiles in conjunction with solar cells to generate electricity, aiming to achieve integration of energy-harvesting components into wearable fabrics.'}

In [51]:
print("response BGE\n :",qa_chain_1("what is new in textiles industry"))
print("response OpenAIEmbedding\n :",qa_chain("what is new in textiles industry"))

response BGE
 : {'query': 'what is new in textiles industry', 'result': 'In the textiles industry, there have been advancements in incorporating photovoltaic technology into fabrics, such as woven and nonwoven fabrics, for applications like energy harvesting. Additionally, there have been developments in using embroidered structures for high-tech applications, including conductive textiles for intelligent sensors. These innovations represent some of the new trends in the textiles industry.'}
response OpenAIEmbedding
 : {'query': 'what is new in textiles industry', 'result': 'One of the newer trends in the textiles industry is the increasing use of embroidered structures for high-tech applications, including those requiring conductive textiles such as intelligent sensors. This advancement provides a contrast to traditional weaving methods and opens up new possibilities for innovative textile products.'}
